### Advanced ML: Домашнее задание 2

Данная работа выполнена в качестве домашнего задания по курсу Advanced Machine Learning в академии больших данных MADE. 

Задача: разработать вероятностнуцю рейтинг систему спортивного "Что? Где? Когда".


Background: в спортивном "Что? Где? Когда?" соревнующиеся команды отвечают на одни и те же вопросы. После минуты обсуждения команды записывают и сдают свои ответы на карточках; побеждает тот, кто ответил на большее число вопросов. Турнир обычно состоит из нескольких десятков вопросов (обычно 36 или 45, иногда 60, больше редко). Часто бывают синхронные турниры, когда на одни и те же вопросы отвечают команды на сотнях игровых площадок по всему миру, т.е. в одном турнире могут играть сотни, а то и тысячи команд. Соответственно, нам нужно:

●	построить рейтинг-лист, который способен нетривиально предсказывать результаты будущих турниров;

●	при этом, поскольку ЧГК — это хобби, и контрактов тут никаких нет, игроки постоянно переходят из команды в команду, сильный игрок может на один турнир сесть поиграть за другую команду и т.д.; поэтому единицей рейтинг-листа должна быть не команда, а отдельный игрок;

●	а что сильно упрощает задачу и переводит её в область домашних заданий на EM-алгоритм — это характер данных: начиная с какого-то момента, в базу результатов начали вносить все повопросные результаты команд, т.е. в данных будут записи вида “какая команда на какой вопрос правильно ответила”.


Ссылка на исходные данные: https://www.dropbox.com/s/s4qj0fpsn378m2i/chgk.zip 

### 1. Загрузка данных

In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from scipy.stats import kendalltau, spearmanr
import math

Загрузим данные об игроках:

In [2]:
with open('players.pkl', 'rb') as fp:
    a = pickle.load(fp)
players = pd.DataFrame.from_dict(a, orient='index')

In [3]:
players.head()

,id,name,patronymic,surname
1,1,Алексей,None,Абабилов
10,10,Игорь,,Абалов
11,11,Наталья,Юрьевна,Абалымова
12,12,Артур,Евгеньевич,Абальян
13,13,Эрик,Евгеньевич,Абальян


In [4]:
players.shape

(204063, 4)

Загрузим данные о турнирах. Выберем турниры 2019 года (тренировочные данные) и 2020 года (тестовые данные)

In [5]:
with open('tournaments.pkl', 'rb') as fp:
    a = pickle.load(fp)
tournaments = pd.DataFrame.from_dict(a, orient='index')

In [6]:
train_tournaments = tournaments[tournaments['dateStart'].str.contains('2019-')]
test_tournaments = tournaments[tournaments['dateStart'].str.contains('2020-')]

In [7]:
train_tournaments.shape, test_tournaments.shape

((687, 9), (418, 9))

In [8]:
train_tournaments.head()

,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
4772,4772,Синхрон северных стран. Зимний выпуск,2019-01-05T19:00:00+03:00,2019-01-09T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 28379, 'name': 'Константин', 'patronym...",{'dateRequestsAllowedTo': '2019-01-09T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
4973,4973,Балтийский Берег. 3 игра,2019-01-25T19:05:00+03:00,2019-01-29T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 23030, 'name': 'Марина', 'patronymic':...",{'dateRequestsAllowedTo': '2019-01-28T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
4974,4974,Балтийский Берег. 4 игра,2019-03-01T19:05:00+03:00,2019-03-05T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 23030, 'name': 'Марина', 'patronymic':...",{'dateRequestsAllowedTo': '2019-03-04T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
4975,4975,Балтийский Берег. 5 игра,2019-04-05T19:05:00+03:00,2019-04-09T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 23030, 'name': 'Марина', 'patronymic':...",{'dateRequestsAllowedTo': '2019-04-08T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
4986,4986,ОВСЧ. 6 этап,2019-02-15T20:00:00+03:00,2019-02-19T20:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 59140, 'name': 'Борис', 'patronymic': ...",{'dateRequestsAllowedTo': '2019-02-19T23:59:59...,"{'1': 12, '2': 12, '3': 12}"


Загрузим результаты игр. 

In [9]:
with open('results.pkl', 'rb') as fp:
    results = pickle.load(fp)

По каждому чемпионату содержится информация обо всех принимающих в нем участие командах в следующем формате:

In [10]:
results[4772][0]

{'team': {'id': 45556,
  'name': 'Рабочее название',
  'town': {'id': 285, 'name': 'Санкт-Петербург'}},
 'mask': '111111111011111110111111111100010010',
 'current': {'name': 'Рабочее название',
  'town': {'id': 285, 'name': 'Санкт-Петербург'}},
 'questionsTotal': 28,
 'synchRequest': {'id': 56392,
  'venue': {'id': 3030, 'name': 'Санкт-Петербург'}},
 'position': 1,
 'controversials': [{'id': 91169,
   'questionNumber': 15,
   'answer': 'Мьёльнир',
   'issuedAt': '2019-01-06T13:28:48+03:00',
   'status': 'A',
   'comment': '',
   'resolvedAt': '2019-01-06T15:25:54+03:00',
   'appealJuryComment': None}],
 'flags': [],
 'teamMembers': [{'flag': 'Б',
   'usedRating': 13507,
   'rating': 13507,
   'player': {'id': 6212,
    'name': 'Юрий',
    'patronymic': 'Яковлевич',
    'surname': 'Выменец'}},
  {'flag': 'Б',
   'usedRating': 10988,
   'rating': 13185,
   'player': {'id': 18332,
    'name': 'Александр',
    'patronymic': 'Витальевич',
    'surname': 'Либер'}},
  {'flag': 'Б',
   'usedRa

### Построение baseline модели

В качестве baseline-модели построим линейную регрессию, которая будет обучать рейтинг-лист игроков. Будем исходить из следующих предположений:

 - пока не будем брать в рассмотрение команды. Результаты всей команды будем относить к каждому игроку индивидуально.
 - при построении модели будем учитывать разную сложность вопросов.
 
Т.о. построим линейную модель, которая принимает на вход ID игрока и ID вопроса и прогнозирует, ответит ли игрок на данный вопрос.

Выгрузка тренировочных данных:

In [11]:
train_ids = list(train_tournaments['id'].values)
train_data = []

for idx in  train_ids:
    
    try:
        num_questions = len(results[idx][0]['mask'])
    except:
        continue
    questions_ids = [str(idx)+'_'+str(i) for i in range(num_questions)]
    num_teams = len(results[idx])
    for team_number in range(num_teams):
        members_ids = [results[idx][team_number]['teamMembers'][i]['player']['id'] for i in range(len(results[idx][team_number]['teamMembers']))]
        team_res = results[idx][team_number]['mask']
        team_id = results[idx][team_number]['team']['id']
        team_name = results[idx][team_number]['team']['name']

        for member in members_ids:
            try:
                for ans in range(len(team_res)):
                    train_data.append([team_id, team_name, member, questions_ids[ans], team_res[ans]])
            except:
                continue

In [12]:
df_train = pd.DataFrame.from_records(train_data, columns = ['team_id', 'team_name', 'player_id', 'question_id', 'target'])
df_train = df_train[(df_train.target == '0') | (df_train.target == '1')]
df_train.target = df_train.target.apply(lambda x: int(x))

In [13]:
df_train

,team_id,team_name,player_id,question_id,target
0,45556,Рабочее название,6212,4772_0,1
1,45556,Рабочее название,6212,4772_1,1
2,45556,Рабочее название,6212,4772_2,1
3,45556,Рабочее название,6212,4772_3,1
4,45556,Рабочее название,6212,4772_4,1
...,...,...,...,...,...
21011666,76130,Ласточки,217156,6255_31,0
21011667,76130,Ласточки,217156,6255_32,0
21011668,76130,Ласточки,217156,6255_33,0
21011669,76130,Ласточки,217156,6255_34,0


player_id и question_id преобразуем в ohe-hot векторы

In [14]:
df_train['player_id'] = df_train['player_id'].astype('category')
X_train = pd.get_dummies(df_train[['player_id', 'question_id']], sparse = True)

In [15]:
X_train

,player_id_15,player_id_16,player_id_23,player_id_31,player_id_35,player_id_38,player_id_47,player_id_59,player_id_65,player_id_79,...,question_id_6255_90,question_id_6255_91,question_id_6255_92,question_id_6255_93,question_id_6255_94,question_id_6255_95,question_id_6255_96,question_id_6255_97,question_id_6255_98,question_id_6255_99
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21011666,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21011667,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21011668,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21011669,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
y_train = df_train.target

In [17]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

В качестве "силы" игрока будем рассматривать коэффициент линейной регрессии, который получился перед соответствующим столбцом в one-hot представлении id игроков.

In [18]:
num_players = len(set(df_train['player_id'].values))

In [19]:
players_power = {}
coef = model.coef_
columns = list(X_train.columns)
for i in range(num_players):
    id_player = columns[i][10:]
    players_power[int(id_player)] = coef[i]

In [20]:
players_power

{15: 0.15879731020073468,
 16: 0.3339393851981636,
 23: 0.23414139301441717,
 31: 0.200911000472507,
 35: 0.20137888282400498,
 38: 0.020440886951727973,
 47: 0.36437738619758503,
 59: 0.22463700444939122,
 65: 0.10254949228573766,
 79: 0.19007581186452946,
 80: 0.37382834046243263,
 82: 0.14512919016168613,
 98: 0.031114249058169265,
 112: 0.14023108682080354,
 113: 0.2112161997401682,
 117: 0.4057660939696919,
 119: 0.18298772278169706,
 133: 0.3523097100132876,
 136: 0.0988357117397055,
 144: 0.5003585653187334,
 150: 0.3570541126282038,
 153: 0.3306857123729609,
 157: 0.10220019772540839,
 160: 0.41664579222622206,
 176: 0.4315133331909495,
 178: 0.2267545104301841,
 182: 0.24789750416714343,
 196: 0.18916085618242434,
 223: 0.4062190055308943,
 230: 0.33777701906205165,
 232: 0.1188264635130177,
 233: 0.11228637117350504,
 236: 0.4506503877391102,
 261: 0.22256302318763074,
 263: 0.1869727234109967,
 278: 0.31699789973618364,
 286: -0.013901965567051455,
 315: 0.383099528851045,
 

### 3. Построение прогнозов результатов турнира, оценка полученных предсказаний.

Нам необходим способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд. В качестве метрики качества на тестовом наборе будем считать ранговые корреляции Спирмена и Кендалла  между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.

Предлагаемый на данном этапе способ ранжирования команд на турнире: суммируем силу каждого игрока, рассчитанную на предущем этапе, и ранжируем по убыванию полученной "суммарной силы". Если данных об игроке не было в обучеющей выборке (он не принимал участие в тернирах 2019 года), то в качестве его "силы" будем считать 0.

In [21]:
test_ids = list(test_tournaments['id'].values) # id турниров 2020 года
test_ids = [idx for idx in test_ids 
            if len(results[idx]) > 1 and
               'mask' in results[idx][0].keys() and
                results[idx][0]['mask'] != None
           ] #по некоторым турнирам отсутствуют данные о результатах. Их удаляем.

In [22]:
print('Размер тестовой выборки получился {} турнира'.format(len(test_ids)))

Размер тестовой выборки получился 172 турнира


In [23]:
known_players = list(players_power.keys()) #игроки, информация по которым была в тренировочной выборке

In [24]:
def get_rating(idx, players_power):
    #idx - id турнира
    #returns - словарь с прогнозными занятыми местами команд по итогам турнира
    num_teams = len(results[idx])
    res = {}
    for team_number in range(num_teams):
        members_ids = [results[idx][team_number]['teamMembers'][i]['player']['id'] 
                       for i in range(len(results[idx][team_number]['teamMembers'])) 
                       if results[idx][team_number]['teamMembers'][i]['player']['id'] in known_players]
        score = np.array([players_power[i] for i in members_ids]).sum()
        res[results[idx][team_number]['team']['id']] = score
    sorted_res = sorted(res.items(), key=lambda x: x[1], reverse=True)
    return {team_id : i + 1 for i, (team_id, _) in enumerate(sorted_res)}

In [25]:
def mean_correlations(list_of_ids, players_power):
    # list_of_ids - список id чемпионатов
    score_spearmanr = []
    score_kendalltau = []
    for idx in list_of_ids:
        rating = get_rating(idx, players_power)
        teams_id = [results[idx][i]['team']['id'] for i in range(len(results[idx]))]
        pred = [rating[i] for i in teams_id]
        target = [results[idx][i]['position'] for i in range(len(results[idx]))]
        score_spearmanr.append(spearmanr(pred, target)[0])
        score_kendalltau.append(kendalltau(pred, target)[0])
    return np.array(score_spearmanr).mean(), np.array(score_kendalltau).mean()

In [26]:
mean_spearmanrmean_corr, mean_kendalltau_corr = mean_correlations(test_ids, players_power)
print('Усредненное значение ранговой корреляции Спирмена = {}'.format(mean_spearmanrmean_corr))
print('Усредненное значение ранговой корреляции Кендалла  = {}'.format(mean_kendalltau_corr))

Усредненное значение ранговой корреляции Спирмена = 0.7811150506207643
Усредненное значение ранговой корреляции Кендалла  = 0.6242398031417601


### 4. Построение ЕМ-алгоритма для предсказания результатов турнира с учетом понимания, что не все игроки команды правильно отвечали на вопрос при том, что команда в целом ответила верно

Реализуем простой ЕМ-алгоритм (на основе запоминающейся лекции про сусликов :) ).
Для каждого игрока с помощью линейной модели мы хотим предсказать вероятность его ответа на конкретный вопрос. Про каждый вопрос нам известно, ответила ли на него команда или нет. Если команда не ответила - значит все понятно: никто из игроков не смог ответить на данный вопрос. А если команда ответила, то мы не знаем, кто конкретно из игроков предложил правильный ответ. 
Т.о. наблюдаемая переменная - ответила ли команда на вопрос. Скрытая переменная (которая нас и интересует) - веротяность того, что на вопрос ответил данный конкретный игрок.

В качестве первого простейшего варианта попробуем следующее:
 - на первом шаге считаем, что ответы каждого игрока совпадают с ответами команды (как в baseline). Как и ранее, модель принимает на вход ID игрока и ID вопроса (в one-hot представлении) а на выходе - число (ответил/не ответил).
 - С помощью этой модели строим прогноз для обучающей выборки. Поскольку мы используем линейную регрессию, в качестве ответа мы получим некоторое вещественное число, которое, в том числе может быть и отрицательным, и большим 1.
 - Для того, чтобы интерпретировать этот результат как вероятность ответа игрока на вопрос, применим к выходу модели сигмоиду. Полученное значение будет таргетом на следующей итерации обучения.
 - Подставляем полученные вероятности вместо таргета (в те вопросы, где команда ответила). Где не ответила - оставляем нули. **Это E-шаг алгоритма**.
 - **М-шаг**: Снова обучаем линейную модель с обновленными на Е-шаге таргетами. 
На каждой итерации силу игрока, как и ранее, будем оценивать как коэффициент полученной линейной модели при 1 в one-hot представлении ID игрока.

In [27]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def obr_sigmoid(x):
    return - np.log(1 / x - 1)

In [28]:
num_epoch = 10
y_current = y_train.copy()
mask = y_train.values == 0
model = LinearRegression()
for epoch in range(num_epoch):
    model.fit(X_train, y_current)
    y_pred = model.predict(X_train)
    y_pred = sigmoid(y_pred)
    np.putmask(y_pred, mask, 0)
    y_current = y_pred
    players_power_curr = {}
    coef = model.coef_
    columns = list(X_train.columns)
    for i in range(num_players):
        id_player = columns[i][10:]
        players_power_curr[int(id_player)] = coef[i]
    mean_spearmanrmean_corr, mean_kendalltau_corr = mean_correlations(test_ids, players_power_curr)
    print('Эпоха {}:'.format(epoch))
    print('Усредненное значение ранговой корреляции Спирмена = {}'.format(mean_spearmanrmean_corr))
    print('Усредненное значение ранговой корреляции Кендалла  = {}'.format(mean_kendalltau_corr))
    print()

Эпоха 0:
Усредненное значение ранговой корреляции Спирмена = 0.7811150506207643
Усредненное значение ранговой корреляции Кендалла  = 0.6242398031417601

Эпоха 1:
Усредненное значение ранговой корреляции Спирмена = 0.7847387288417692
Усредненное значение ранговой корреляции Кендалла  = 0.6287096889050325

Эпоха 2:
Усредненное значение ранговой корреляции Спирмена = 0.7848934890563253
Усредненное значение ранговой корреляции Кендалла  = 0.6290261688495393

Эпоха 3:
Усредненное значение ранговой корреляции Спирмена = 0.7848683168701835
Усредненное значение ранговой корреляции Кендалла  = 0.6288987166445178

Эпоха 4:
Усредненное значение ранговой корреляции Спирмена = 0.7848482953471194
Усредненное значение ранговой корреляции Кендалла  = 0.6289028742948803

Эпоха 5:
Усредненное значение ранговой корреляции Спирмена = 0.7848551535498984
Усредненное значение ранговой корреляции Кендалла  = 0.6289134013887692

Эпоха 6:
Усредненное значение ранговой корреляции Спирмена = 0.7848559477551773
Ус

Видим, что наша целевая метрика увеличивается, хоть и крайне медленными темпами.

В реализованном выше алгоритме никак не учитывался состав конкретной команды. Для каждого игрока исходы (ответит или не ответит игрок на конкретный вопрос) были независимы от результатов других игроков. Однако, на самом деле, это не так.  
Нам необходимо оценить апостериорную вероятность случайного события $x_{i}$ - игрок $x$ правильно ответил на вопрос $i$. Пусть $z_{i} = 1$, если команда игрока $x$ ответила на вопрос $i$, $z_{i} = 0$, если команда не ответила. Тогда:
$$ p(x_{i}|z_{i}) = p(x_{i}|z_{i} = 1) + p(x_{i}|z_{i} = 0)$$.
$ p(x_{i}|z_{i} = 0)$ всегда равна 0, так как если бы игрок $x$ ответил на вопрос, то и команда в целом ответила бы.
Тогда:
$$ p(x_{i}|z_{i}) = p(x_{i}|z_{i} = 1) = \frac{p(x, z = 1)}{p(z = 1)}$$

Т.о. предлагаемый EM алгоритм:
 - Как и ранее, на первом шаге предполагаем, что ответы каждого игрока совпадают с ответами команды в целом.
 - На основании этого строим линейную регрессию, предсказывающую вероятность игрока ответить на данный вопрос (в случае, если команда в целом ответила. Если не ответила - то вероятность ответа данного игрока = 0). Это мы получили совместную вероятность 
    $p(x_{i}, z_{i} = 1)$. 
 - Далее рассчитываем условную вероятность:
 $$p(x_{i}|z_{i} = 1) = \frac{p(x, z = 1)}{p(z = 1)},$$
где:
$$p(z = 1) = 1 - \prod\limits_{n = 1}^{N}(1-p(y_{n})),$$
где $p(y_{n})$ - вероятность того, что $n$-ый  член команды ответил на вопрос, $N$ - количество игроков в команде.
 - На следующем шаге полученные условные вероятности подставляем в качестве таргетов в линейную модель и повторяем процесс итеративно.

In [29]:
num_epoch = 10
y_current = y_train.copy()
mask = y_train.values == 0
model = LinearRegression()
for epoch in range(num_epoch):
    model.fit(X_train, y_current)
    y_pred = model.predict(X_train)
    y_pred = sigmoid(y_pred)
    np.putmask(y_pred, mask, 0)
    
    df_current = df_train.copy()
    df_current['incorrect_ans_prob'] = 1 - y_pred #вероятности, что НЕ ответит на вопрос 
    df_group = df_current.groupby(by = ['team_id', 'question_id'])['incorrect_ans_prob'].prod().reset_index()
    df_group['correct_answer_prob_team'] = 1 - df_group['incorrect_ans_prob']
    df_current = df_current.merge(df_group[['team_id', 'question_id', 'correct_answer_prob_team']], on = ['team_id', 'question_id'])
    df_current['new_target'] = (1 - df_current['incorrect_ans_prob']) / df_current['correct_answer_prob_team']
      
    df_current.replace(np.nan, 0, inplace = True)
    
    y_current = df_current['new_target'].values
    
    players_power_curr = {}
    coef = model.coef_
    columns = list(X_train.columns)
    for i in range(num_players):
        id_player = columns[i][10:]
        players_power_curr[int(id_player)] = coef[i]
    mean_spearmanrmean_corr, mean_kendalltau_corr = mean_correlations(test_ids, players_power_curr)
    print('Эпоха {}:'.format(epoch))
    print('Усредненное значение ранговой корреляции Спирмена = {}'.format(mean_spearmanrmean_corr))
    print('Усредненное значение ранговой корреляции Кендалла  = {}'.format(mean_kendalltau_corr))
    print()

Эпоха 0:
Усредненное значение ранговой корреляции Спирмена = 0.7811150506207643
Усредненное значение ранговой корреляции Кендалла  = 0.6242398031417601

Эпоха 1:
Усредненное значение ранговой корреляции Спирмена = 0.7820293928964164
Усредненное значение ранговой корреляции Кендалла  = 0.6253753356338532

Эпоха 2:
Усредненное значение ранговой корреляции Спирмена = 0.783372251402335
Усредненное значение ранговой корреляции Кендалла  = 0.6265843084679521

Эпоха 3:
Усредненное значение ранговой корреляции Спирмена = 0.7834777529199717
Усредненное значение ранговой корреляции Кендалла  = 0.6266876346495789

Эпоха 4:
Усредненное значение ранговой корреляции Спирмена = 0.7835099389717741
Усредненное значение ранговой корреляции Кендалла  = 0.6267712895654471

Эпоха 5:
Усредненное значение ранговой корреляции Спирмена = 0.7835151562989222
Усредненное значение ранговой корреляции Кендалла  = 0.6267829593507593

Эпоха 6:
Усредненное значение ранговой корреляции Спирмена = 0.783515388054666
Усре

Видим, что наша метрика сошлась к значению, немного большему, чем в baseline. Но значение метрик получилось ниже, чем в первом варинате EM алгоритма, где мы не учитывали зависимость вероятностей ответа членов одной команды. Это неожиданный результат...

### 5. Рейтинг-лист турниров по сложности вопросов

Постройм “рейтинг-лист” турниров по сложности вопросов. В качестве меры сложности отдельного вопроса примем коэффициент при соответствующем столбце в матрице признаков. Сложность будет обратно пропорциональна значению коэффициента, т.е. наиболее сложные вопросы - те, которые имеют максимальный по модулю отрицательный коэффициент.
Сложность турнира определим как среднее значение сложности вопросов (так как количество вопросов в разный турнирах может отличаться).

In [30]:
questions_difficulty = []
coef = model.coef_
columns = list(X_train.columns)
for i in range(num_players, len(columns)):
    id_tournament, id_question = columns[i][12:].split('_')
    questions_difficulty.append([int(id_tournament), int(id_question), coef[i]])
questions_diff_data = pd.DataFrame(questions_difficulty, columns = ['id' , 'id_question', 'difficulty'])
questions_diff_data = questions_diff_data.groupby(by = 'id')['difficulty'].mean().reset_index()
train_train_tournaments_difficulty = train_tournaments.merge(questions_diff_data, on = 'id')
train_train_tournaments_difficulty = train_train_tournaments_difficulty.sort_values(by = 'difficulty')
train_train_tournaments_difficulty.head(10)

,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty,difficulty
662,6149,Чемпионат Санкт-Петербурга. Первая лига,2019-10-13T00:00:00+03:00,2019-12-01T15:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/53,"[{'id': 26469, 'name': 'Алексей', 'patronymic'...",None,"{'1': 15, '2': 15, '3': 15, '4': 15, '5': 15, ...",-0.317921
540,5928,Угрюмый Ёрш,2019-11-21T14:00:00+03:00,2019-11-27T14:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/53,"[{'id': 21952, 'name': 'Павел', 'patronymic': ...",{'dateRequestsAllowedTo': '2019-11-26T23:55:00...,"{'1': 15, '2': 15, '3': 15}",-0.237326
368,5684,Синхрон высшей лиги Москвы,2019-06-14T19:00:00+03:00,2019-06-18T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 38458, 'name': 'Екатерина', 'patronymi...",{'dateRequestsAllowedTo': '2019-06-17T23:59:59...,"{'1': 12, '2': 12, '3': 12}",-0.203409
638,6101,Воображаемый музей,2019-12-05T00:01:00+03:00,2019-12-11T23:59:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/53,"[{'id': 111748, 'name': 'Андрей', 'patronymic'...",{'dateRequestsAllowedTo': '2019-12-11T23:59:00...,"{'1': 12, '2': 12, '3': 12}",-0.203224
43,5159,Первенство правого полушария,2019-08-09T19:30:00+03:00,2019-08-13T19:30:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 39254, 'name': 'Артём', 'patronymic': ...",{'dateRequestsAllowedTo': '2019-08-13T23:59:59...,"{'1': 12, '2': 12, '3': 12}",-0.200028
376,5693,Знание – Сила VI,2019-08-16T19:00:00+03:00,2019-08-20T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 36120, 'name': 'Серафим', 'patronymic'...",{'dateRequestsAllowedTo': '2019-08-20T23:59:59...,"{'1': 12, '2': 12, '3': 12}",-0.185407
283,5587,Записки охотника,2019-04-13T12:00:00+03:00,2019-04-17T12:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 84551, 'name': 'Игорь', 'patronymic': ...",{'dateRequestsAllowedTo': '2019-04-17T23:59:59...,"{'1': 12, '2': 12, '3': 12}",-0.183401
26,5083,Ускользающая сова,2019-01-04T14:00:00+03:00,2019-01-08T14:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 37584, 'name': 'Иван', 'patronymic': '...",{'dateRequestsAllowedTo': '2019-01-06T23:59:59...,"{'1': 12, '2': 12, '3': 12}",-0.183171
551,5942,Чемпионат Мира. Этап 2. Группа В,2019-09-07T17:00:00+03:00,2019-09-07T19:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/53,"[{'id': 27247, 'name': 'Александр', 'patronymi...",None,"{'1': 15, '2': 15}",-0.181512
178,5465,Чемпионат России,2019-05-18T12:00:00+03:00,2019-05-19T18:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/52,"[{'id': 31038, 'name': 'Владимир', 'patronymic...",None,"{'1': 15, '2': 15, '3': 15, '4': 15, '5': 15, ...",-0.180188


В целом, в самые сложные чемпионаты попали Второй этап чемпионата мира (правда, только 9 место), Чемпионат России (10 место), Чемпионат Санкт-Петербурга. Первая лига (1 место по сложности).

Посмотрим на самые легкие чемпионаты:

In [31]:
train_train_tournaments_difficulty.tail(10)

,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty,difficulty
171,5457,Студенческий чемпионат Калининградской области,2019-02-16T14:00:00+03:00,2019-02-16T19:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/52,"[{'id': 90340, 'name': 'Дмитрий', 'patronymic'...",None,"{'1': 15, '2': 15, '3': 15}",0.245643
562,5954,Школьная лига. II тур.,2019-11-08T19:00:00+03:00,2020-05-01T19:00:00+03:00,"{'id': 8, 'name': 'Асинхрон'}",/seasons/53,"[{'id': 39218, 'name': 'Владислав', 'patronymi...",{'dateRequestsAllowedTo': '2019-11-17T23:59:00...,"{'1': 12, '2': 12, '3': 12}",0.250312
7,5009,(а)Синхрон-lite. Лига старта. Эпизод III,2019-01-25T14:00:00+03:00,2019-02-25T23:55:00+03:00,"{'id': 8, 'name': 'Асинхрон'}",/seasons/52,"[{'id': 23740, 'name': 'Владимир', 'patronymic...",{'dateRequestsAllowedTo': '2019-02-23T23:59:59...,"{'1': 12, '2': 12, '3': 12}",0.250954
379,5698,(а)Синхрон-lite. Лига старта. Эпизод VII,2019-09-01T00:05:00+03:00,2019-09-30T23:55:00+03:00,"{'id': 8, 'name': 'Асинхрон'}",/seasons/53,"[{'id': 23740, 'name': 'Владимир', 'patronymic...",{'dateRequestsAllowedTo': '2019-09-29T23:59:59...,"{'1': 12, '2': 12, '3': 12}",0.256708
672,6254,Школьная лига,2019-10-04T19:00:00+03:00,2020-03-22T19:00:00+03:00,"{'id': 5, 'name': 'Общий зачёт'}",/seasons/53,"[{'id': 39218, 'name': 'Владислав', 'patronymi...",None,"{'1': 36, '2': 36, '3': 36, '4': 36, '5': 36, ...",0.257177
10,5012,Школьный Синхрон-lite. Выпуск 2.5,2019-04-05T12:00:00+03:00,2019-05-05T23:55:00+03:00,"{'id': 8, 'name': 'Асинхрон'}",/seasons/52,"[{'id': 23740, 'name': 'Владимир', 'patronymic...",{'dateRequestsAllowedTo': '2019-05-03T23:59:59...,"{'1': 12, '2': 12, '3': 12}",0.258232
563,5955,Школьная лига. III тур.,2019-12-06T19:00:00+03:00,2020-05-01T19:00:00+03:00,"{'id': 8, 'name': 'Асинхрон'}",/seasons/53,"[{'id': 39218, 'name': 'Владислав', 'patronymi...",{'dateRequestsAllowedTo': '2019-12-15T23:59:00...,"{'1': 12, '2': 12, '3': 12}",0.262319
545,5936,Школьная лига. I тур.,2019-10-04T19:00:00+03:00,2020-05-01T19:00:00+03:00,"{'id': 8, 'name': 'Асинхрон'}",/seasons/53,"[{'id': 39218, 'name': 'Владислав', 'patronymi...",{'dateRequestsAllowedTo': '2019-10-13T23:59:00...,"{'1': 12, '2': 12, '3': 12}",0.266718
153,5438,Синхрон Лиги Разума,2019-02-02T00:55:00+03:00,2019-02-06T00:40:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 97261, 'name': 'Ярослав', 'patronymic'...",{'dateRequestsAllowedTo': '2019-02-04T23:59:59...,"{'1': 12, '2': 12, '3': 12}",0.269343
11,5013,(а)Синхрон-lite. Лига старта. Эпизод V,2019-04-05T12:00:00+03:00,2019-05-05T23:55:00+03:00,"{'id': 8, 'name': 'Асинхрон'}",/seasons/52,"[{'id': 23740, 'name': 'Владимир', 'patronymic...",{'dateRequestsAllowedTo': '2019-05-03T23:59:59...,"{'1': 12, '2': 12, '3': 12}",0.278929


Видим, что среди самых легких чемпионатов в основном школьные и студенческие турниры. В целом, это неплохо соответствует интуиции, что на турнирах более высокого уровня должны быть более сложные вопросы.

Хотелось бы посмотреть на тексты наиболее сложнных вопросов, но, к сожалению, в базе вопросов внесены далеко не все чемпионаты, названия и даты чемпионатов отличаются от тех, что в датасете. Поэтому, к сожалению, не удалось найти интересующие вопросы.

### Анализ соответствия предсказанной силы игрока и количества игр, в которых он принимал участие.

Построим топ игроков по предсказанной моделью силе игры, а рядом с именами игроков отразим число вопросов, которое они сыграли. Несмотря на то, что первый вариант EM-алгоритма (с независимыми вероятностями игроков) показал чуть более высокие значения целевых меток, будем рассматривать второй алгоритм, так как он кажется более корректным с вероятностной точки зрения.

In [32]:
players_activity = df_train.groupby(by='player_id')['question_id'].count().reset_index()
players_activity.rename(columns={'question_id': 'question_count'}, inplace=True)

def get_leaderboard(players_power):
    players_power_data =  pd.DataFrame(players_power.items(), columns = ['player_id', 'power'])
    players_power_data = players_power_data.merge(players_activity, on='player_id')
    players_power_data.rename(columns={'player_id': 'id'}, inplace = True)
    players_power_data = players.merge(players_power_data, on = 'id')
    players_power_data = players_power_data.sort_values(by = 'power', ascending = False)
    
    return players_power_data

In [33]:
players_power_data = get_leaderboard(players_power_curr)
players_power_data.head(20).reset_index(drop = True)

,id,name,patronymic,surname,power,question_count
0,197185,Дарья,Сергеевна,Зверева,0.544029,36
1,197183,Евгений,Юрьевич,Берхман,0.540609,36
2,16586,Игорь,Олегович,Кружалов,0.539189,36
3,24342,Денис,Владимирович,Пахомов,0.520796,69
4,199350,Роман,Валерьевич,Теренин,0.508125,36
5,191331,Екатерина,,Чигулина,0.498760,35
6,139511,Михаил,Сергеевич,Ганущак,0.497274,24
7,17750,Галина,Вячеславовна,Лазарева,0.495655,36
8,707,Елена,Андреевна,Александрова,0.490422,48
9,195430,Антон,Владимирович,Грачёв,0.486532,36


Из топ-20 игроков (в соответствии с предсказаниями нашей модели) только один человек - Брутер Александра Владимировна - ответила более 100 вопросов (то есть сыграла более 2 игр). Кстати говоря, ее позиция в официпльном рейтинге на сайте "Что? Где? Когда?" - 23. То есть в отношении ее прогноз нашей модели оказался достаточно близким. Остальные игроки, попавшие в топ-20 нашей модели, ответили менее, чем на 100 вопросов, т.е. приняли участие всего в 1-2 играх.

Это естественное свойство модели: за счёт EM-схемы влияние 1-2 удачно сыгранных турниров будет только усиливаться, потому что неудачных турниров, чтобы его компенсировать, у этих игроков нет. Более того, это не мешает метрикам качества, потому что если эти игроки сыграли всего 1-2 турнира в 2019-м, скорее всего они ничего или очень мало сыграли и в 2020, и их рейтинги никак не влияют на качество тестовых предсказаний. Но для реального рейтинга такое свойство, конечно, крайне нежелательно. Попробуем его исправить.

Попробуем убрать из расчетов рейтингов игроков, сыгравших слишком мало турниров. Судя по полученной статистике, медиана количества вопросов - 108 (3 игры). Среднее количество игр за год - 9-10. 
Попробуем сначала просто обнулить вероятности правильных ответов для всех игроков, попытавшихся ответов менее чем на 108 вопрос.

In [34]:
df_train_with_count = df_train.merge(players_activity, on='player_id', how = 'left')
thresh = 108
mask_question_count = df_train_with_count.question_count.values < thresh

In [35]:
df_train_with_count

,team_id,team_name,player_id,question_id,target,question_count
0,45556,Рабочее название,6212,4772_0,1,3432
1,45556,Рабочее название,6212,4772_1,1,3432
2,45556,Рабочее название,6212,4772_2,1,3432
3,45556,Рабочее название,6212,4772_3,1,3432
4,45556,Рабочее название,6212,4772_4,1,3432
...,...,...,...,...,...,...
20908139,76130,Ласточки,217156,6255_31,0,72
20908140,76130,Ласточки,217156,6255_32,0,72
20908141,76130,Ласточки,217156,6255_33,0,72
20908142,76130,Ласточки,217156,6255_34,0,72


In [36]:
num_epoch = 5
thresh = 108

y_train = df_train_with_count.target
y_current = y_train.copy()
mask = y_train.values == 0
model = LinearRegression()
for epoch in range(num_epoch):
    model.fit(X_train, y_current)
    y_pred = model.predict(X_train)
    y_pred = sigmoid(y_pred)
    np.putmask(y_pred, mask, 0)
    np.putmask(y_pred, mask_question_count, 0)
    
    df_current = df_train_with_count.copy()
    df_current['incorrect_ans_prob'] = 1 - y_pred #вероятности, что НЕ ответит на вопрос 
    df_group = df_current.groupby(by = ['team_id', 'question_id'])['incorrect_ans_prob'].prod().reset_index()
    df_group['correct_answer_prob_team'] = 1 - df_group['incorrect_ans_prob']
    df_current = df_current.merge(df_group[['team_id', 'question_id', 'correct_answer_prob_team']], on = ['team_id', 'question_id'])
    df_current['new_target'] = (1 - df_current['incorrect_ans_prob']) / df_current['correct_answer_prob_team']
      
    df_current.replace(np.nan, 0, inplace = True)
    
    y_current = df_current['new_target'].values
    
    players_power_curr = {}
    coef = model.coef_
    columns = list(X_train.columns)
    for i in range(num_players):
        id_player = columns[i][10:]
        players_power_curr[int(id_player)] = coef[i]
    mean_spearmanrmean_corr, mean_kendalltau_corr = mean_correlations(test_ids, players_power_curr)
    print('Эпоха {}:'.format(epoch))
    print('Усредненное значение ранговой корреляции Спирмена = {}'.format(mean_spearmanrmean_corr))
    print('Усредненное значение ранговой корреляции Кендалла  = {}'.format(mean_kendalltau_corr))
    print()

Эпоха 0:
Усредненное значение ранговой корреляции Спирмена = 0.7811150506207643
Усредненное значение ранговой корреляции Кендалла  = 0.6242398031417601

Эпоха 1:
Усредненное значение ранговой корреляции Спирмена = 0.7738374117544914
Усредненное значение ранговой корреляции Кендалла  = 0.6165445082963745

Эпоха 2:
Усредненное значение ранговой корреляции Спирмена = 0.7752160281172963
Усредненное значение ранговой корреляции Кендалла  = 0.6182231261158864

Эпоха 3:
Усредненное значение ранговой корреляции Спирмена = 0.7748675214158293
Усредненное значение ранговой корреляции Кендалла  = 0.6179313905183711

Эпоха 4:
Усредненное значение ранговой корреляции Спирмена = 0.7748619665509773
Усредненное значение ранговой корреляции Кендалла  = 0.6179381518979536



Видим, что качество предсказаний заметно снизилось. Попробуем другой вариант: если количество вопросов игрока менее порога, не обнулим вероятность, а умножим на коэффициент "Количество вопросов, сыгранных игроком / Порог".

In [37]:
def probability_growth(x, thresh):
    if x < thresh:
        return x / thresh
    else:
        return 1

In [38]:
thresh = 108
num_epoch = 5
y_train = df_train_with_count.target
y_current = y_train.copy()
mask = y_train.values == 0
model = LinearRegression()
for epoch in range(num_epoch):
    model.fit(X_train, y_current)
    y_pred = model.predict(X_train)
    y_pred = sigmoid(y_pred)
    np.putmask(y_pred, mask, 0)
    #np.putmask(y_pred, mask_question_count, 0)
    
    df_current = df_train_with_count.copy()
    df_current['incorrect_ans_prob'] = 1 - y_pred #вероятности, что НЕ ответит на вопрос 
    df_group = df_current.groupby(by = ['team_id', 'question_id'])['incorrect_ans_prob'].prod().reset_index()
    df_group['correct_answer_prob_team'] = 1 - df_group['incorrect_ans_prob']
    df_current = df_current.merge(df_group[['team_id', 'question_id', 'correct_answer_prob_team']], on = ['team_id', 'question_id'])
    df_current['new_target'] = (1 - df_current['incorrect_ans_prob']) / df_current['correct_answer_prob_team']
    
    df_current['probability_growth'] = df_current['question_count'].apply(lambda x: probability_growth(x, thresh))
    df_current['new_target'] = df_current['new_target'] * df_current['probability_growth']
    
    df_current.replace(np.nan, 0, inplace = True)
    
    y_current = df_current['new_target'].values
    
    players_power_curr = {}
    coef = model.coef_
    columns = list(X_train.columns)
    for i in range(num_players):
        id_player = columns[i][10:]
        players_power_curr[int(id_player)] = coef[i]
    mean_spearmanrmean_corr, mean_kendalltau_corr = mean_correlations(test_ids, players_power_curr)
    print('Эпоха {}:'.format(epoch))
    print('Усредненное значение ранговой корреляции Спирмена = {}'.format(mean_spearmanrmean_corr))
    print('Усредненное значение ранговой корреляции Кендалла  = {}'.format(mean_kendalltau_corr))
    print()

Эпоха 0:
Усредненное значение ранговой корреляции Спирмена = 0.7811150506207643
Усредненное значение ранговой корреляции Кендалла  = 0.6242398031417601

Эпоха 1:
Усредненное значение ранговой корреляции Спирмена = 0.7813521087674323
Усредненное значение ранговой корреляции Кендалла  = 0.6240206660236877

Эпоха 2:
Усредненное значение ранговой корреляции Спирмена = 0.7827565579584322
Усредненное значение ранговой корреляции Кендалла  = 0.6251596646863691

Эпоха 3:
Усредненное значение ранговой корреляции Спирмена = 0.7829881613063823
Усредненное значение ранговой корреляции Кендалла  = 0.6253719304603749

Эпоха 4:
Усредненное значение ранговой корреляции Спирмена = 0.7829203766540473
Усредненное значение ранговой корреляции Кендалла  = 0.6253442618269243



Видим, что качество предсказаний осталось примерно на том же уровне, что и до корректировки на количество игр. Посмотрим на получившийся рейтинг:

In [39]:
players_power_data = get_leaderboard(players_power_curr)
players_power_data.head(20).reset_index(drop = True)

,id,name,patronymic,surname,power,question_count
0,24342,Денис,Владимирович,Пахомов,0.507428,69
1,4270,Александра,Владимировна,Брутер,0.474562,3007
2,707,Елена,Андреевна,Александрова,0.471821,48
3,28751,Иван,Николаевич,Семушин,0.467172,4089
4,30152,Артём,Сергеевич,Сорожкин,0.464010,5218
5,27822,Михаил,Владимирович,Савченков,0.455438,3656
6,19060,Александр,Александрович,Людикайнен,0.453407,295
7,27403,Максим,Михайлович,Руссо,0.451931,2457
8,31701,Алексей,Сергеевич,Титов,0.447757,108
9,74382,Михаил,Андреевич,Новосёлов,0.445332,3265


Видим, что в топе оказалось уже большее количество игроков, сыгравших значительное количество игр. 

Попробуем теперь скорректировать рвероятность ответа игрока следующим образом: возьмем пороговое значение вопросов thresh и применим к нему функцию:
$$f(x) = tanh(\frac{x}{thresh}).$$
при этом для совсем мальнького количества вопросов корректировка будет достаточно значительной, а для игроков, которые сыграли более чем $2 * thresh$ вопросов изменения рассчитанного рейтинга будут минимальны.

In [40]:
df_train_with_count['normal_question_count'] = df_train_with_count['question_count'].apply(lambda x: math.tanh(x / thresh))

In [41]:
df_train_with_count

,team_id,team_name,player_id,question_id,target,question_count,normal_question_count
0,45556,Рабочее название,6212,4772_0,1,3432,1.000000
1,45556,Рабочее название,6212,4772_1,1,3432,1.000000
2,45556,Рабочее название,6212,4772_2,1,3432,1.000000
3,45556,Рабочее название,6212,4772_3,1,3432,1.000000
4,45556,Рабочее название,6212,4772_4,1,3432,1.000000
...,...,...,...,...,...,...,...
20908139,76130,Ласточки,217156,6255_31,0,72,0.582783
20908140,76130,Ласточки,217156,6255_32,0,72,0.582783
20908141,76130,Ласточки,217156,6255_33,0,72,0.582783
20908142,76130,Ласточки,217156,6255_34,0,72,0.582783


In [42]:
thresh = 108
num_epoch = 5
y_train = df_train_with_count.target
y_current = y_train.copy()
mask = y_train.values == 0
model = LinearRegression()
for epoch in range(num_epoch):
    model.fit(X_train, y_current)
    y_pred = model.predict(X_train)
    y_pred = sigmoid(y_pred)
    np.putmask(y_pred, mask, 0)
    
    df_current = df_train_with_count.copy()
    
    df_current['ans_prob'] = y_pred
    df_current['ans_prob'] = df_current['ans_prob'] * df_current['normal_question_count']
    df_current['incorrect_ans_prob'] = 1 - df_current['ans_prob']
     
    df_group = df_current.groupby(by = ['team_id', 'question_id'])['incorrect_ans_prob'].prod().reset_index()
    df_group['correct_answer_prob_team'] = 1 - df_group['incorrect_ans_prob']
    df_current = df_current.merge(df_group[['team_id', 'question_id', 'correct_answer_prob_team']], on = ['team_id', 'question_id'])
    df_current['new_target'] = (1 - df_current['incorrect_ans_prob']) / df_current['correct_answer_prob_team'] 
      
    df_current.replace(np.nan, 0, inplace = True)
    
    y_current = df_current['new_target'].values
    
    players_power_curr = {}
    coef = model.coef_
    columns = list(X_train.columns)
    for i in range(num_players):
        id_player = columns[i][10:]
        players_power_curr[int(id_player)] = coef[i]
    mean_spearmanrmean_corr, mean_kendalltau_corr = mean_correlations(test_ids, players_power_curr)
    print('Эпоха {}:'.format(epoch))
    print('Усредненное значение ранговой корреляции Спирмена = {}'.format(mean_spearmanrmean_corr))
    print('Усредненное значение ранговой корреляции Кендалла  = {}'.format(mean_kendalltau_corr))
    print()

Эпоха 0:
Усредненное значение ранговой корреляции Спирмена = 0.7811150506207643
Усредненное значение ранговой корреляции Кендалла  = 0.6242398031417601

Эпоха 1:
Усредненное значение ранговой корреляции Спирмена = 0.7816936979623964
Усредненное значение ранговой корреляции Кендалла  = 0.6239968050485664

Эпоха 2:
Усредненное значение ранговой корреляции Спирмена = 0.782081336169943
Усредненное значение ранговой корреляции Кендалла  = 0.6247864567163355

Эпоха 3:
Усредненное значение ранговой корреляции Спирмена = 0.7820194675475602
Усредненное значение ранговой корреляции Кендалла  = 0.6247623085819022

Эпоха 4:
Усредненное значение ранговой корреляции Спирмена = 0.7819769998323817
Усредненное значение ранговой корреляции Кендалла  = 0.6247352375878662



Качество предсказаний практически не изменилось. Посмотрим на получившийся теперь рейтинг игроков:

In [43]:
players_power_data = get_leaderboard(players_power_curr)
players_power_data.head(20).reset_index(drop = True)

,id,name,patronymic,surname,power,question_count
0,24342,Денис,Владимирович,Пахомов,0.498923,69
1,4270,Александра,Владимировна,Брутер,0.473019,3007
2,707,Елена,Андреевна,Александрова,0.469866,48
3,28751,Иван,Николаевич,Семушин,0.465626,4089
4,30152,Артём,Сергеевич,Сорожкин,0.462387,5218
5,27822,Михаил,Владимирович,Савченков,0.453897,3656
6,27403,Максим,Михайлович,Руссо,0.450354,2457
7,19060,Александр,Александрович,Людикайнен,0.450145,295
8,74382,Михаил,Андреевич,Новосёлов,0.443740,3265
9,18332,Александр,Витальевич,Либер,0.440614,4093


ТОП-20 рейтинга остался практически без изменений, но некоторые игроки поменялись местами по сравнению с предыдущим вариантом.
Посмотрим на официальный рейтинг на сайте "Что? Где? Когда" тех игроков, которые сыграли более 1000 вопросов и все равно попали в топ-10:
- Брутер Александра Владимировна - 32 место,
- Семушин Иван Николаевич - 28 место,
- Сорожкин Артем Сергеевич - 1 место,
- Савченков Михаил Владимирович - 6,5 место,
- Руссо Максим Михайлович - 28 место,
- Новосёлов Михаил Андреевич - 123 место,
- Либер Александр Витальевич - 10 место,

То есть игроки, сыгравшие более 1000 игр и попавшие в топ-10 рассчитанного нами рейтинга, действительно имеют высокий рейтинг в игре.
В качестве эксперимента пробовала менять пороговое значение (например, 1000 игр) - результаты остались практически такими же.

### Выводы и комментарии

В данной работе опробованы различные варианты реализации EM-алгоритма для построения вероятностной модели для прогнозирования результатов турниров спортивного "Что? где? когда?". Оказалось, что предсказательная способность всех вариантов алгоритма практически одинакова и совсем незначительно превосходит baseline, в котором вообще не учитывалась "командность" данной игры. Тем не менее, полученный итоговый рейтинг игроков, сыгравших более 1000 вопросов в 2019 году, коррелирует с официальными данными в игре. 